In [1]:
import os

In [2]:
%pwd

'c:\\Projekte\\Data_Science\\End-to-end-Machine-Learning-Project-with-MLflow\\Git_Project\\MLFlow-Machine-Learning\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Projekte\\Data_Science\\End-to-end-Machine-Learning-Project-with-MLflow\\Git_Project\\MLFlow-Machine-Learning'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    transformation_config: dict
    all_schema: dict
    target: dict

In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        transformation_config = self.params.data_transformation
        schema = self.schema.COLUMNS
        target = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            transformation_config=transformation_config,
            all_schema=schema,
            target = target
        )

        return data_transformation_config

In [8]:
import os
import joblib
from mlProject import logger

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [9]:
class TrainTestSplitterSK:

    def __init__(self, config, inference):
        self.config = config
        self.inference = inference
        for transformation in config.transformation_config:
            if transformation.get('name') == 'TrainTestSplitterSK':
                self.test_size = transformation.get("test_size")

    def transform_data(self, data):

        # Split the data into training and test sets. (0.75, 0.25) split.
        if not self.inference:

            train, test = train_test_split(data)

            train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
            test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

            logger.info("Splitted data into training and test sets")
            logger.info(train.shape)
            logger.info(test.shape)

            print(train.shape)
            print(test.shape)


class StandardScalerSK:

    def __init__(self, config, inference):
        self.config = config
        self.inference = inference
        for transformation in config.transformation_config:
            if transformation.get("name") == "StandardScalerSK":
                self.with_mean = transformation.get("with_mean")

    def transform_data(self, data):
        if self.inference:
            scaler = joblib.load(os.path.join(self.config.root_dir, "scaler.pkl"))
            data = scaler.transform(data)
            data.to_csv(os.path.join(self.config.root_dir, "data.csv"), index=False)
        else:
            # Split the data into training and test sets. (0.75, 0.25) split.
            scaler = StandardScaler()
            train = pd.read_csv(os.path.join(self.config.root_dir, "train.csv"))
            test = pd.read_csv(os.path.join(self.config.root_dir, "test.csv"))
            train_target = train[self.config.target.name]
            test_target = test[self.config.target.name]
            train_scaled = pd.DataFrame(
                scaler.fit_transform(train.drop(columns=[self.config.target.name])),
                columns=train.drop(columns=[self.config.target.name]).columns,
            )
            test_scaled = pd.DataFrame(
                scaler.transform(test.drop(columns=[self.config.target.name])),
                columns=test.drop(columns=[self.config.target.name]).columns,
            )

            # Add the "death" column back
            train = pd.concat([train_scaled, train_target], axis=1)
            test = pd.concat([test_scaled, test_target], axis=1)
            train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
            test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)
            joblib.dump(scaler, os.path.join(self.config.root_dir, "scaler.pkl"))

            logger.info("Scaled the data")

In [10]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig, inference=False):
        self.config = config
        self.inference = inference

    def apply_transformations(self):
        data = pd.read_csv(self.config.data_path)
        for transformation in self.config.transformation_config:
            # Create an instance of the class
            class_name = transformation.name
            if class_name in globals():
                Transformator = globals()[class_name](config=self.config, inference=self.inference)
                Transformator.transform_data(data=data)
            else:
                raise NameError(f"Class {class_name} is not defined.")

        return data


In [11]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.apply_transformations()
except Exception as e:
    raise e

[2024-09-20 09:11:12,186: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-20 09:11:12,189: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-20 09:11:12,194: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-09-20 09:11:12,197: INFO: common: created directory at: artifacts]
[2024-09-20 09:11:12,201: INFO: common: created directory at: artifacts/data_transformation]
[2024-09-20 09:11:12,226: INFO: 1496529258: Splitted data into training and test sets]
[2024-09-20 09:11:12,227: INFO: 1496529258: (224, 13)]
[2024-09-20 09:11:12,228: INFO: 1496529258: (75, 13)]
(224, 13)
(75, 13)
[2024-09-20 09:11:12,302: INFO: 1496529258: Scaled the data]
